In [ ]:
import os
import re

# Modifications

### Load data:

In [ ]:
# !wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
# !wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip

In [ ]:
# !wget https://cv.snu.ac.kr/research/EDSR/benchmark.tar
# !wget https://cv.snu.ac.kr/research/EDSR/benchmark.tar

### Unzip downloads:

In [ ]:
# %mkdir div2k
# !unzip -q DIV2K_train_HR.zip -d div2k
# !unzip -q DIV2K_train_LR_bicubic_X2.zip -d div2k
# !tar -xf benchmark.tar

### Make symlinks (necessary for BasicSR):

In [1]:
# %mkdir basicsr/datasets
# %mkdir basicsr/datasets/DIV2K
# %mkdir basicsr/datasets/benchmark

In [2]:
# root = !pwd
# root = root[0]

# !ln -s $root/div2k/*  $root/basicsr/datasets/DIV2K
# !ln -s  $root/benchmark/*  $root/basicsr/datasets/benchmark

### Make patches:

In [ ]:
# %cd basicsr
# !python3 ../scripts/data_preparation/extract_subimages.py
# %cd ..

### Change parameters:

In [ ]:
train_params = {
    'num_gpu': 1,
    'num_worker_pre_gpu': 4,
    'batch_size_per_gpu': 16,
    'dataset_enlarge_ratio': 1,
    'total_iter': 10000,
    'val_freq': '!!float 1e3',
    'save_checkpoint_freq': '!!float 5e3',
    'print_freq': 100,
    'use_tb_logger': 'false'
}

In [ ]:
test_params = {
    'num_gpu': 1
}

In [ ]:
def get_model_name(filename):
    parts = filename.split('.')[0].split('_')[1:]
    new_parts = [parts[0]]
    for part in parts[1:]:
        if part.lower() == 'se':
            new_parts.append('SE') 
            continue
        new_parts.append(part.capitalize()) 
    return "".join(new_parts)

In [ ]:
root = os.path.abspath('.')
train_path = 'options/train/'

for file in os.listdir(train_path):
    full_path = root + '/' + train_path + file
    text = None
    with open(full_path, 'r') as f:
        text = f.read()
        text = re.sub('dataroot_gt: [a-zA-Z0-9-/]+/datasets/', 'dataroot_gt: ' + root + '/basicsr/datasets/', text)
        text = re.sub('dataroot_lq: [a-zA-Z0-9-/]+/datasets/', 'dataroot_lq: ' + root + '/basicsr/datasets/', text)
        for param, value in train_params.items():
            text = re.sub(param + ':' + '.*\n', param + ': ' + str(value) + '\n', text)
    text = re.sub('name:' + '.*\n', 'name: ' + file.split('.')[0] + '\n', text, count=1)
    text = re.sub('network_g:\n  type: ' + '.*\n', 'network_g:\n  type: ' + get_model_name(file) + '\n', text, count=1)
    with open(full_path, 'w') as f:
        f.write(text)

In [ ]:
root = os.path.abspath('.')
test_path = 'options/test/'

for file in os.listdir(test_path):
    full_path = root + '/' + test_path + file
    text = None
    with open(full_path, 'r') as f:
        text = f.read()
        text = re.sub('dataroot_gt: [a-zA-Z0-9-/]+/datasets/', 'dataroot_gt: ' + root + '/basicsr/datasets/', text)
        text = re.sub('dataroot_lq: [a-zA-Z0-9-/]+/datasets/', 'dataroot_lq: ' + root + '/basicsr/datasets/', text)
        for param, value in test_params.items():
            text = re.sub(param + ':' + '.*\n', param + ': ' + str(value) + '\n', text)
    text = re.sub('name:' + '.*\n', 'name: ' + file.split('.')[0] + '\n', text, count=1)
    text = re.sub('network_g:\n  type: ' + '.*\n', 'network_g:\n  type: ' + get_model_name(file) + '\n', text, count=1)
    text = re.sub(
        'pretrain_network_g: .*\n', 
        'pretrain_network_g: ' + root + '/experiments/' + \
            file.replace('test', 'train').split('.')[0] + '/models/net_g_latest.pth' + '\n', 
        text
    )
    with open(full_path, 'w') as f:
        f.write(text)

## Experiments:

In [ ]:
%cd basicsr/

### EDSR

In [ ]:
# !python3 train.py -opt ../options/train/train_EDSR.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_EDSR.yaml

### Binary EDSR

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR.yaml

### SE-attention on binary branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_none_se.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_none_se.yaml

### Spatial attention on binary branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_none_spat.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_none_spat.yaml

### Pixel attention on binary branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_none_pixel.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_none_pixel.yaml

### SE-attention on residual branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_se_none.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_se_none.yaml

### Spatial attention on residual branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_spat_none.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_spat_none.yaml

### Pixel attention on residual branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_pixel_none.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_pixel_none.yaml

### SE-attention on both branches

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_se_se.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_se_se.yaml

### SE-attention on residual branch, spatial attention on binary branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_se_spat.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_se_spat.yaml

### SE-attention on residual branch, pixel attention on binary branch

In [ ]:
# !python3 train.py -opt ../options/train/train_BinEDSR_se_pixel.yaml

In [ ]:
# !python3 test.py -opt ../options/test/test_BinEDSR_se_pixel.yaml